# **⚖️ EMPLOYMENT LAW: CASE LAW DISCOVERY SYATEM UNDER KENYA LAW**

## **🎯 PROJECT OBJECTIVE**

* ### To build a smart legal research tool that helps advocates quickly find accurate, relevant, and authoritative employment law cases from Kenya Law based on legal issues described in plain English.

## **🧩 PROJECT SCOPE**

### Area:

* ### Employment & Labour Relations

### Courts:

* ### Employment and Labour Relations Court (ELRC)

* ### Court of Appeal (employment cases)

* ### Supreme Court (where applicable)

### Jurisdiction:

* ### Kenya

### Output:

* ### Research assistance only (NOT legal advice)

## **METHODOLOGY**

STEP 1: DATA COLLECTION (KENYA LAW)

STEP 2: LEGAL TEXT PREPROCESSING

STEP 3: EMBEDDINGS (CORE INTELLIGENCE)

STEP 4: SEMANTIC SEARCH (THE MAGIC)

STEP 5: RELEVANCE & AUTHORITY RANKING

STEP 6: DISPLAY MEANINGFUL RESULTS

STEP 7: USER INTERFACE (STREAMLIT)

STEP 8: LEGAL & ETHICAL SAFEGUARDS

# **DATA COLLECTION FROM KENYA LAW**

## STEP 1: MANUALLY COLLECT JUDGMENT LINKS

In [14]:
# Importing libraries

import requests
from bs4 import BeautifulSoup
import os
import pandas as pd
import time
import re
import json

import torch
from sentence_transformers import SentenceTransformer

import warnings
warnings.filterwarnings('ignore')

In [15]:
base_url = "https://new.kenyalaw.org/akn/ke/judgment/keelrc/2025/962/eng@2025-03-27"

In [16]:
headers = {
    "User-Agent": "EmploymentLawResearchBot/1.0 (Academic Project)"
}

response = requests.get(base_url, headers=headers, timeout=20)
response.raise_for_status()

soup = BeautifulSoup(response.text, "lxml")

## STEP 2: EXTRACT CORE METADATA

In [17]:
# Case title
title = soup.find("h1")
case_title = title.get_text(strip=True) if title else "Unknown Title"

# Court
court_tag = soup.find("meta", {"name": "court"})
court = court_tag["content"] if court_tag else "ELRC"

# Date
date_tag = soup.find("meta", {"name": "date"})
decision_date = date_tag["content"] if date_tag else "Unknown Date"

print(case_title, court, decision_date)

Kitui v Winguard Services Ltd (Miscellaneous Application E263 of 2024) [2025] KEELRC 962 (KLR) (27 March 2025) (Ruling) ELRC Unknown Date


## STEP 3: EXTRACT THE JUDGMENT BODY

In [18]:
for tag in soup.find_all(["article", "section", "div"]):
    text = tag.get_text(strip=True)
    if len(text) > 1000:
        print(tag.name, tag.get("class"))


div ['pb-5']
div ['tab-content', 'mt-3']
div ['tab-pane', 'fade', 'show', 'active']
div ['container']
div ['container-fluid']
div ['document-content']
div ['document-content__inner']
div ['content-and-enrichments']
div ['content-and-enrichments__inner', 'la-akoma-ntoso-with-gutter']
article ['akn-judgment']
div ['akn-judgmentBody']
div ['akn-arguments']
div ['akn-paragraph']


In [19]:
judgment_div = soup.find(
    lambda tag: tag.name in ["div", "article", "section"]
    and tag.get_text(strip=True)
    and len(tag.get_text(strip=True)) > 2000
)

if not judgment_div:
    raise ValueError("Judgment body not found")

judgment_text = judgment_div.get_text(separator="\n", strip=True)


In [20]:
print("===== JUDGMENT PREVIEW =====")
print(judgment_text[:700])
print("\n===== END PREVIEW =====")


===== JUDGMENT PREVIEW =====
Home
Judgments
Superior Courts
Employment and Labour Relations Court
Employment and Labour Relations Court at Nairobi
Kitui v Winguard Services Ltd (Miscellaneous Application E263 of 2024) [2025] KEELRC 962 (KLR) (27 March 2025) (Ruling)
Download DOCX
          (66.1 KB)
Toggle dropdown
Download PDF
Report
Report a problem
Kitui v Winguard Services Ltd (Miscellaneous Application E263 of 2024) [2025] KEELRC 962 (KLR) (27 March 2025) (Ruling)
Copy citation
Document detail
Citations
1 / -
Citation
Kitui v Winguard Services Ltd [2025] KEELRC 962 (KLR)
Copy
Media Neutral Citation
[2025] KEELRC 962 (KLR)
Copy
Court
Employment and Labour Relations Court
Court station
Employment and Labour Relations

===== END PREVIEW =====


# **CLEAN LEGAL TEXT TO IMPROVE QUALITY**

In [21]:
def clean_akn_text(text):
    text = re.sub(r'\n+', '\n', text)
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

clean_text = clean_akn_text(judgment_text)

# **SAVE STRUCTURED OUTPUT (BEST PRACTICE)**

In [22]:
os.makedirs("data/akn_cases", exist_ok=True)

case_data = {
    "case_title": case_title,
    "court": court,
    "decision_date": decision_date,
    "source_url": base_url,
    "text": clean_text
}

file_name = "keelrc_2025_962.json"

with open(f"data/akn_cases/{file_name}", "w", encoding="utf-8") as f:
    json.dump(case_data, f, ensure_ascii=False, indent=2)

# **SPLIT INTO SEARCHABLE CHUNKS (FOR NLP)**

In [23]:
def chunk_text(text, chunk_size=500):
    words = text.split()
    chunks = []
    for i in range(0, len(words), chunk_size):
        chunks.append(" ".join(words[i:i + chunk_size]))
    return chunks

chunks = chunk_text(clean_text)

# **CREATE EMBEDDINGS**

In [25]:
model = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = model.encode(chunks)


In [26]:
print(embeddings.shape)

(4, 384)
